In [1]:
import sys
sys.path.insert(0,'..')
from llmdh.story_llms import *

In [2]:
fn='/Users/ryan/lltk_data/corpora/canon_fiction/txt/chadwyck.Austen.Emma_A_Novel_In_Three_Vol.1090.txt'
# fn='/Users/ryan/lltk_data/corpora/canon_fiction/txt/chadwyck.Haywood.Betsy_Thoughtless.1038.txt'
# fn='/Users/ryan/lltk_data/corpora/canon_fiction/txt/chadwyck.Burney.Evelina.1008.txt'
# fn='/Users/ryan/lltk_data/corpora/canon_fiction/txt/chadwyck.Dickens.Bleak_House_By_Charles_Di.1168.txt'
with open(fn) as f: txt=f.read()
len(txt.split())

160778

In [3]:
system_prompt="""
You are a literary critic. Your task is to extract and describe all romantic relationships in a novel, whether actual or suggested as potential. 

For each actual or potential romantic relationship between characters, determine:
1) The name, gender, class, annual income (if explicitly stated), and a description of both characters involved in the relationship
2) Whether the characters were already in a relationship at the start of the novel
3) Whether the characters ended up in a relationship by the end of the novel
4) A numerical estimate of the likelihood of a relationship forming, in the range of 0 to 10. For example, if two matches for a character are suggested as potential, and one of the is a dashing Mr Truelove and another a contemptible Mr Fool, the reader can suspect that Mr Trueworth will be the eventual match (likelihood 9), and not Mr Fool (likelihood 1)
5) The nature of the eventual relationship (whether marriage or unmarried partnership)
6) If the relationship ends, determine the manner in which it ended (e.g. divorce, death of one or both characters, etc)
7) A short summary of the relationship's beginning, middle, and end

Return a list of JSON dictionaries of the following form:

[
    {
        "character1_name": "Elizabeth Bennet",
        "character2_name": "Mr Darcy",
        "character1_gender": "Female",
        "character2_gender": "Male",
        "character1_class": "gentry",
        "character2_class": "aristocrat",
        "character1_income": "£2,000 (her father, Mr Bennet's income)",
        "character2_income": "£10,000",
        "character1_desc": "Elizabeth Bennet is the intelligent and spirited second daughter of a moderately wealthy landed gentleman.",
        "character2_desc": "Mr Darcy is a wealthy and initially aloof gentleman of high social standing.",
        "relationship_at_start": false,
        "relationship_by_end": true,
        "relationship_likelihood": 7,
        "relationship_is_marriage": true,
        "relationship_ended": false,
        "relationship_ended_reason": "",
        "relationship_summary": "Elizabeth meets Mr Darcy at a ball, where she overhears him insulting him; Elizabeth harbors contempt for him, and rejects his first offer of marriage; but she warms to him when he demonstrates his selfless love by helping her resolve a scandal involving her sister, Lydia; she accepts his second, humbler of marriage."
    },
    
]

"""

In [4]:
res4=LLM.generate_gemini(txt, system_prompt=system_prompt, model='gemini-1.5-pro-latest')

In [5]:
data=dirty_jsonl_loads(res4)
pprint(data)

[{'character1_class': 'governess',
  'character1_desc': 'Miss Taylor is a kind, intelligent, and well-respected '
                     'woman who served as governess and close friend to Emma '
                     'Woodhouse. She is described as even-tempered, selfless, '
                     "and devoted to Emma's well-being.",
  'character1_gender': 'Female',
  'character1_income': None,
  'character1_name': 'Miss Taylor',
  'character2_class': 'gentry',
  'character2_desc': 'Mr. Weston is a pleasant, good-humored, and sociable man '
                     'of comfortable fortune. He is a widower with a grown '
                     'son, Frank, who was raised by wealthy relatives.',
  'character2_gender': 'Male',
  'character2_income': None,
  'character2_name': 'Mr. Weston',
  'relationship_at_start': False,
  'relationship_by_end': True,
  'relationship_ended': False,
  'relationship_ended_reason': '',
  'relationship_is_marriage': True,
  'relationship_likelihood': 10,
  'relationsh

In [6]:
pd.DataFrame(data).sort_values('relationship_likelihood',ascending=False)

,character1_name,character2_name,character1_gender,character2_gender,character1_class,character2_class,character1_income,character2_income,character1_desc,character2_desc,relationship_at_start,relationship_by_end,relationship_likelihood,relationship_is_marriage,relationship_ended,relationship_ended_reason,relationship_summary
0,Miss Taylor,Mr. Weston,Female,Male,governess,gentry,None,None,"Miss Taylor is a kind, intelligent, and well-r...","Mr. Weston is a pleasant, good-humored, and so...",False,True,10,True,False,,Emma plays matchmaker and encourages the relat...
4,Jane Fairfax,Frank Churchill,Female,Male,dependent orphan,gentry,None,None,"Jane Fairfax is a beautiful, accomplished, and...","Frank Churchill is Mr. Weston's son, a handsom...",True,True,10,True,False,,Jane and Frank are secretly engaged from the s...
3,Emma Woodhouse,Frank Churchill,Female,Male,gentry,gentry,"£2,000 (father's)",None,"Emma Woodhouse is the intelligent, beautiful, ...","Frank Churchill is Mr. Weston's son, a handsom...",False,False,7,None,None,None,Frank Churchill's arrival in Highbury causes a...
2,Emma Woodhouse,Mr. Elton,Female,Male,gentry,clergyman,"£2,000 (father's)",None,"Emma Woodhouse is the intelligent, beautiful, ...",Mr. Elton is the handsome and ambitious vicar ...,False,False,6,None,None,None,Emma misinterprets Mr. Elton's attentions towa...
1,Harriet Smith,Robert Martin,Female,Male,unknown (likely lower gentry),yeoman farmer,None,None,"Harriet Smith is a pretty, sweet-tempered, but...",Robert Martin is a respectable and intelligent...,False,True,5,True,False,,"Robert Martin proposes to Harriet, but she is ..."
6,Harriet Smith,Mr. Knightley,Female,Male,unknown (likely lower gentry),gentry,None,None,"Harriet Smith is a pretty, sweet-tempered, but...","Mr. Knightley is a sensible, kind, and discern...",False,False,4,None,None,None,Emma mistakenly believes Mr. Knightley is attr...
5,Emma Woodhouse,Mr. Knightley,Female,Male,gentry,gentry,"£2,000 (father's)",None,"Emma Woodhouse is the intelligent, beautiful, ...","Mr. Knightley is a sensible, kind, and discern...",False,True,2,True,False,,Emma and Mr. Knightley share a long-standing f...
